# 검증세트   
test set으로 성능을 올바르게 예측하려면 가능한 test set을 사용하지 않아야 한다. 마지막에 딱 한번 사용하는 것이 좋다.   
그렇다면 test set을 사용하지 않고 하이퍼파라미터 튜닝을 어떻게 할 것인가?   
바로 train set을 또 나누면 된다. 이 나눈 데이터를 validation set이라고 한다.   

train set으로 훈련하고 validation set으로 모델을 평가한다. 그리고 마지막에 둘을 합쳐 최종 모델을 다시 훈련한다.

In [2]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine-date')

In [3]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine[['class']].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 폴드 교차 검증.   
train set을 세부분 나누어서 교차 검증을 수행하는 것.

In [19]:
#사이킷런의 cross_validate() 라는 교차 검증 함수를 사용한다. 기본적으로 5폴드 교차검증.
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) #모델객체, train set을 매개변수로 전달한다.
print(scores)

{'fit_time': array([0.00630403, 0.00458932, 0.        , 0.        , 0.01566577]), 'score_time': array([0., 0., 0., 0., 0.]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [20]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [26]:
#cross_validate()는 train set을 섞어서 폴더를 나누지 않는다. 섞기 위해서 분할기 splitter를 지정해야 한다.
#cross_validate()는 기본적으로 회귀모델일경우 KFold분할기를 사용하고, 분류모델일 경우 StratifiedKFold를 사용한다.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [28]:
#10폴드 교차검증 하기.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
score = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(score['test_score']))

0.8574181117533719


# 그리드서치(하이퍼파라미터 튜닝하기)   
GridSearchCV클래스는 하이퍼파라미터 탐색과 교차검증을 한 번에 수행한다.


In [29]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}#최적의 값을 찾아보고 싶은 하이퍼 파라미터.

In [36]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #GridSearchCV는 cv 매개변수의 기본값은 5이다.

In [39]:
gs.fit(train_input, train_target) #훈련하면 결정트리모델의 min_impurity_decrease값을 바꿔가며 총 5번 실행한다. 5x5 = 25개의 모델 훈련.

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [38]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [45]:
#GridSearchCV는 검증점수가 가장 높은 모델의 매개변수 조합으로 전체 train set에서 자동으로 다시 훈련한다.
#다시 훈련된 모델은 gs객체의 best_estimator_ 속성에 저장된다.
dt = gs.best_estimator_
print(dt.score(test_input, test_target))


0.8653846153846154


In [46]:
#GridSearchCV로 찾은 최적의 하이퍼파라미터 값은 best_params_에 저장된다.
print(gs.best_params_)

#각 하이퍼파라미터에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장되어있다.
print(gs.cv_results_['mean_test_score'])

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [48]:
#argmax()함수를 사용하여 가장 큰 값의 인덱스를 추출하여, 이 인덱스값을 사용해 params키에 저장된 매개변수를 출력할 수 있다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [53]:
#여러개의 하이퍼 파라미터 값 구하기.
#min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량
#max_depth : 트리의 깊이
#min_sample_split : 노드를 나누기 위한 최소 샘플 수.
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
         'max_depth' : range(5,20,1),
         'min_samples_split' : range(2,100,10)}

In [54]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [55]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [56]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤서치   
매개변수 값의 목록을 전달하는 것이 아닌 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달한다.

In [60]:
#싸이파이라이브러리사용.
from scipy.stats import uniform, randint #두 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.(균등분포 샘플링)
#randint는 정수값을 뽑고, uniform은 실수값을 뽑는다.

In [63]:
#ex> radint
rgen = randint(0,10)
rgen.rvs(10)

#1000개를 뽑아 균등하게 샘플링 되었는지 확인하기.
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92,  86, 117,  96,  96, 104, 119, 112,  91,  87], dtype=int64))

In [64]:
#ex> uniform
ugen = uniform(0,1)
ugen.rvs(10)

array([0.04041007, 0.42569929, 0.55349118, 0.13333179, 0.87615783,
       0.05897816, 0.94825212, 0.78790391, 0.7058201 , 0.02762525])

In [65]:
#mean_sample_leaf : 리프노드가 되기 위한 최소 샘플의 개수.
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth' : randint(20,50),
         'min_samples_split' : randint(2,25),
         'min_samples_leaf' : randint(1,25)}

In [69]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000129869093A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000012986909580>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000129855A5E50>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000129855A5D30>},
                   random_state=42)

In [70]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [71]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [72]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
